In [ ]:
# Boilerplate that all notebooks reuse:
from analysis_common import *

%matplotlib inline

# Kernel analysis

In [ ]:
df = read_ods("./results.ods", "histogram-kernel")
expand_modes(df)

order=['DRAM', 'AD', 'MM (hot)', 'MM (cold)']
hue_order = [100000000, 2000000]

sns.barplot(x='MODE', y='TIMING',
            data=df[(df.POINTS_PER_FRAGMENT == 2000000)],
            capsize=0.1,
            order=order,
            palette=custom_kernel_palette(4))
plt.title("2 million values per fragment (small object)")
plt.ylabel("kernel exec time (s)")
plt.show()

sns.barplot(x='MODE', y='TIMING',
            data=df[(df.POINTS_PER_FRAGMENT == 100000000)],
            capsize=0.1,
            order=order,
            palette=custom_kernel_palette(4))
plt.title("100 million values per fragment (big object)")
plt.ylabel("kernel exec time (s)")
plt.show()

In [ ]:
df.loc[(df.POINTS_PER_FRAGMENT == 2000000), "NORMALIZED"] = df.TIMING 
df.loc[(df.POINTS_PER_FRAGMENT == 100000000), "NORMALIZED"] = df.TIMING / 50

ax = sns.barplot(y='MODE', x='NORMALIZED',
                 data=df,
                 capsize=0.1,
                 order=order,
                 hue_order=hue_order,
                 hue="POINTS_PER_FRAGMENT",
                 palette="muted")

kernel_plot_tweaks(ax, 50, legend_title="Values per fragment")
plt.savefig("histogram-kernel.pdf", bbox_inches='tight')
plt.show()


In [ ]:
kernel_times = df.groupby(["POINTS_PER_FRAGMENT", "MODE"]).min()
kernel_times


# Histogram results analysis

In [ ]:
df = read_ods("./results.ods", "histogram-app")
expand_modes(df)

In [ ]:
for ppf in [2000000, 100000000]:
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "DRAM"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(ppf, "DRAM"), "TIMING"]
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "AD"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(ppf, "AD"), "TIMING"]
    df.loc[(df.POINTS_PER_FRAGMENT == ppf) & (df.MODE == "DAOS"), "ATOM_KERNEL"] =  \
        kernel_times.loc[(ppf, "DRAM"), "TIMING"]

df.loc[(df.POINTS_PER_FRAGMENT == 2000000) 
       & (df.NUMBER_OF_FRAGMENTS == 1600) 
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(2000000, "MM (hot)"), "TIMING"]
df.loc[(df.POINTS_PER_FRAGMENT == 100000000) 
       & (df.NUMBER_OF_FRAGMENTS == 32)
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(100000000, "MM (hot)"), "TIMING"]
df.loc[(df.POINTS_PER_FRAGMENT == 2000000) 
       & (df.NUMBER_OF_FRAGMENTS == 12800) 
       & (df.MODE == "MM"),
       "ATOM_KERNEL"] = kernel_times.loc[(2000000, "MM (cold)"), "TIMING"]
df.loc[(df.POINTS_PER_FRAGMENT == 100000000) 
       & (df.NUMBER_OF_FRAGMENTS == 256) 
       & (df.MODE == "MM"), 
       "ATOM_KERNEL"] = kernel_times.loc[(100000000, "MM (cold)"), "TIMING"]

df["KERNEL_TIME"] = df["NUMBER_OF_FRAGMENTS"] * df["ATOM_KERNEL"]

# Sanity check
null_values = df[df.isnull().values]
if len(null_values) > 0:
    null_values # show that

In [ ]:
sns.set(style="whitegrid")

small = (
    ((df.POINTS_PER_FRAGMENT == 2000000) & (df.NUMBER_OF_FRAGMENTS == 1600)) |
    ((df.POINTS_PER_FRAGMENT == 100000000) & (df.NUMBER_OF_FRAGMENTS == 32))
)

big = (
    ((df.POINTS_PER_FRAGMENT == 2000000) & (df.NUMBER_OF_FRAGMENTS == 12800)) |
    ((df.POINTS_PER_FRAGMENT == 100000000) & (df.NUMBER_OF_FRAGMENTS == 256))
)

order = ["DRAM", "AD", "MM", "DAOS"]

ax = sns.barplot(y='MODE', x="TIMING",
                 data=df[small],
                 capsize=0.1,
                 order=order,
                 hue_order=hue_order,
                 palette="colorblind",
                 hue=df.POINTS_PER_FRAGMENT)

bottom = sns.barplot(y='MODE', x="KERNEL_TIME",
            data=df[small],
            capsize=0,
            errwidth=0,
            order=order,
            hue_order=hue_order,
            palette="pastel",
            hue=df.POINTS_PER_FRAGMENT)

ylabel_tweaks(ax, [1, 3], ['non-active', 'active'], 0.16, 0.005)
legend_tweaks(bottom, ["big objects", "small objects", "kernel comp."], placement='upper center')
plt.xlabel("execution time (s)")
plt.title("Small dataset")
save_tweaks('histogram-small.pdf')
plt.show()

ax = sns.barplot(y='MODE', x="TIMING",
                 data=df[big],
                 capsize=0.1,
                 order=order,
                 hue_order=hue_order,
                 palette="colorblind",
                 hue=df.POINTS_PER_FRAGMENT)

annotate_dram(ax)

bottom = sns.barplot(y='MODE', x="KERNEL_TIME",
            data=df[big],
            capsize=0,
            errwidth=0,
            order=order,
            hue_order=hue_order,
            palette="pastel",
            hue=df.POINTS_PER_FRAGMENT)

ylabel_tweaks(ax, [1, 3], ['non-active', 'active'], 0.16, 0.005)
legend_tweaks(bottom, ["big objects", "small objects", "kernel comp."], placement='upper center')
plt.xlabel("execution time (s)")
plt.title("Big dataset")
save_tweaks('histogram-big.pdf')
plt.show()

In [ ]:
df.groupby(["POINTS_PER_FRAGMENT", "NUMBER_OF_FRAGMENTS", "MODE"]).mean()